# Streaming With LangChain

Streaming is critical in making applications based on LLMs feel responsive to end-users.

Important LangChain primitives like LLMs, parsers, prompts, retrievers, and agents implement the LangChain [Runnable Interface](/docs/expression_language/interface).

This interface provides two general approaches to stream content for your application:

1. sync `stream` and async `astream`: a **default implementation** of streaming that streams the **final output** from the chain.
2. async `astream_events` and async`astream_log`: these provide a way to stream both **intermediate steps** and **final output** from the chain.

Here, we'll take a look at both approaches, and try to understand a bit of what's happening under the hood. 🥷

## Using Stream

All `Runnable` objects implement a sync method called `stream` and an async variant called `astream`. 

These methods are designed to stream the final output in chunks, yielding each chunk as soon as it is available.

Streaming is only possible if all steps in the program are able to operate directly on an **input stream**, process an input chunk one at a time, and yield a corresponding output chunk.

Such logic can range from being trivial (e.g., output the tokens generated by the LLM) to fairly difficult (e.g., streaming partial JSON results before the full JSON is available).

The best place to start is with the simplest components, so we can see how streaming works with them!

### LLMs and Chat Models

Large language models and their chat variants are the primary bottleneck in LLM based apps. 🙊

Large language models may take up to a **few seconds** to generate a complete response to a query.

This is far larger than the **~200-300 ms** threshold at which an application still feels responsive to an end user.

The primary solution to this problem is to stream the output from the model **token by token**.

In [244]:
from langchain.chat_models import ChatAnthropic
model = ChatAnthropic()

chunks = []
async for chunk in model.astream('hello. tell me something about yourself'):
    chunks.append(chunk)
    print(chunk.content, end="|")

 Hello|!| My| name| is| Claude|.| I|'m| an| artificial| intelligence| assistant| created| by| An|throp|ic| to| be| helpful|,| harmless|,| and| honest|.||

Let's inspect one of the chunks

In [245]:
chunks[0]

AIMessageChunk(content=' Hello')

We got back something called an `AIMessageChunk`. This chunk represents a part of an `AIMessage`.

Message chunks are additive by design -- one can simply add them up to get the state of the response so far!

In [247]:
chunks[0] + chunks[1] + chunks[2] + chunks[3] + chunks[4]

AIMessageChunk(content=' Hello! My name is')

### Chains

Virtually all LLM applications involve more steps than just a call to a language model.

Let's build a simple chain using `LangChain Expression Language` (`LCEL`) that combines a prompt, model and a parser and verify that streaming works.

We will use `StrOutputParser` to parse the output from the model. This is a simple parser that extracts the `content` field from an `AIMessageChunk`, giving us the `token` returned by the model.

In [241]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
parser = StrOutputParser()
chain = prompt | model | parser

async for chunk in chain.astream({'topic': 'parrot'}):
    print(chunk, end='|')

|Sure|,| here|'s| a| par|rot| joke| for| you|:

|Why| did| the| par|rot| wear| a| rain|coat|?

|Because| it| wanted| to| be| a| "|p|olly|"| uns|aturated|!||

:::{.callout-tip}
`LangChain Expression Language` provides a **declarative** way to specify a "program" by chainining together LangChain primitives. 

Chains created using LCEL benefit from an automatic implementation of `stream`, and `astream` allowing streaming of final output. (In fact, such chains implement
the entire standard Runnable interface.)

You do not have to use the `LangChain Expression Language` to use LangChain and can instead rely on a standard **imperative** programming approach by
caling `invoke`, `batch` or `stream` on each component individually, assigning the results to variables and then using them downstream as you see fit.

You won't get the benefits of using LCEL -- but this might just fine for your needs 👌!
:::

### Working with Input Streams

What if you wanted to stream JSON from the output as it was being generated?

If you were to rely on `json.loads` to parse the partial json, the parsing would fail as the partial json wouldn't be valid json.

Instead, the parser needs to operate on the **input stream**, and attempt to extract valid json from the partial model output.

**note** at the time of writing this parser only works well with OpenAI output.


### Model + JsonParser



In [210]:
from langchain_core.output_parsers import JsonOutputParser

chain = model | JsonOutputParser()

async for text in chain.astream('output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries"'):
    print(text)

{}
{'countries': []}
{'countries': [{}]}
{'countries': [{'name': ''}]}
{'countries': [{'name': 'France'}]}
{'countries': [{'name': 'France', 'population': ''}]}
{'countries': [{'name': 'France', 'population': '67'}]}
{'countries': [{'name': 'France', 'population': '67,'}]}
{'countries': [{'name': 'France', 'population': '67,081'}]}
{'countries': [{'name': 'France', 'population': '67,081,'}]}
{'countries': [{'name': 'France', 'population': '67,081,000'}]}
{'countries': [{'name': 'France', 'population': '67,081,000'}, {}]}
{'countries': [{'name': 'France', 'population': '67,081,000'}, {'name': ''}]}
{'countries': [{'name': 'France', 'population': '67,081,000'}, {'name': 'Spain'}]}
{'countries': [{'name': 'France', 'population': '67,081,000'}, {'name': 'Spain', 'population': ''}]}
{'countries': [{'name': 'France', 'population': '67,081,000'}, {'name': 'Spain', 'population': '46'}]}
{'countries': [{'name': 'France', 'population': '67,081,000'}, {'name': 'Spain', 'population': '46,'}]}
{'co

### Runnable Lambdas as Generators

An important thing to remember is that all parts of the chain must be able to operate on **input streams** rather than on `inputs` or the final result will not stream properly!

Let's use the previous example and add a bit of logic on top that breaks streaming using a function that extracts the countries names from the JSON.

:::{.callout-tip}
Even a chain contains steps that do not operate on input streams, `astream_events` (described below) will yield streaming results from intermediate steps, which may be just fine for your application!
:::

In [213]:
from langchain_core.output_parsers import JsonOutputParser

def _extract_country_names(inputs):
    """A function that does not operates on input streams and breaks streaming."""
    if not isinstance(inputs, dict):
        return ''

    if 'countries' not in inputs:
        return ''

    countries = inputs['countries']

    if not isinstance(countries, list):
        return ''

    country_names = [
        country.get('name')
        for country in countries
        if isinstance(country, dict)
    ]
    return country_names
  

chain = model | JsonOutputParser() | _extract_country_names

async for text in chain.astream('output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries"'):
    print(text, end='|')

['France', 'Spain', 'Japan']|

:::{.callout-tip}
Using a generator function (a function that uses `yield`) allows writing code that operators on **input streams**
:::

In [218]:
from langchain_core.output_parsers import JsonOutputParser

async def _extract_country_names(input_stream):
    """A function that operates on input streams."""
    country_names_so_far = set()

    async for input in input_stream:
        if not isinstance(input, dict):
            continue

        if 'countries' not in input:
            continue

        countries = input['countries']

        if not isinstance(countries, list):
            continue

        for country in countries:
            name = country.get('name')
            if not name:
                continue
            if name not in country_names_so_far:
                yield name
                country_names_so_far.add(name)

chain = model | JsonOutputParser() | _extract_country_names

async for text in chain.astream('output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries"'):
    print(text, end='|')

France|Spain|Japan|

### Components without .stream?

Some components like Retrievers 

Retrievers are not able to stream their results currently and they do not operate on partial input 🤨. 

In [230]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho", 
    "harrison likes spicy food"], 
     embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

chunks = [chunk for chunk in retriever.stream('where did harrison work?')]
chunks

[[Document(page_content='harrison worked at kensho'),
  Document(page_content='harrison likes spicy food')]]

This is OK 🥹! Streaming doesn't have to be implemented for all components as in some cases it may be impossible or effectivley operate on the input stream.

An LCEL chain constructed using 


In [232]:
retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [236]:
for chunk in retrieval_chain.stream("Where did harrison work? Write 3 made up sentences about this place."):
    print(chunk, end='|')

|H|arrison| worked| at| Kens|ho|,| a| renowned| technology| company| known| for| its| innovative| solutions|.| The| company|'s| headquarters| were| located| in| a| bustling| city|,| offering| a| vibrant| work| environment|.| Kens|ho| was| known| for| its| collaborative| culture|,| where| employees| were| encouraged| to| think| outside| the| box| and| push| the| boundaries| of| technology|.| The| office| space| at| Kens|ho| was| modern| and| sleek|,| equipped| with| state|-of|-the|-art| facilities| to| foster| creativity| and| productivity|.||

## Using Stream Events

### Model

In [83]:
events = []
async for event in model.astream_events('hello', version='v1'):
    events.append(event)

In [86]:
events[:3]

[{'event': 'on_chat_model_start',
  'run_id': '652436d1-1ad1-4560-817b-a92981f6a995',
  'name': 'ChatOpenAI',
  'tags': [],
  'metadata': {},
  'data': {'input': 'hello'}},
 {'event': 'on_chat_model_stream',
  'run_id': '652436d1-1ad1-4560-817b-a92981f6a995',
  'tags': [],
  'metadata': {},
  'name': 'ChatOpenAI',
  'data': {'chunk': AIMessageChunk(content='')}},
 {'event': 'on_chat_model_stream',
  'run_id': '652436d1-1ad1-4560-817b-a92981f6a995',
  'tags': [],
  'metadata': {},
  'name': 'ChatOpenAI',
  'data': {'chunk': AIMessageChunk(content='Hello')}}]

In [88]:
events[-2:]

[{'event': 'on_chat_model_stream',
  'run_id': '652436d1-1ad1-4560-817b-a92981f6a995',
  'tags': [],
  'metadata': {},
  'name': 'ChatOpenAI',
  'data': {'chunk': AIMessageChunk(content='')}},
 {'event': 'on_chat_model_end',
  'name': 'ChatOpenAI',
  'run_id': '652436d1-1ad1-4560-817b-a92981f6a995',
  'tags': [],
  'metadata': {},
  'data': {'output': AIMessageChunk(content='Hello! How can I assist you today?')}}]

### Models with prompt

In [28]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

async for chunk in chain.astream({'topic': 'parrot'}):
    print(chunk.content, end='|')

|Sure|,| here|'s| a| par|rot| joke| for| you|:

|Why| don|'t| scientists| trust| par|rots|?

|Because| they| always| talk| in| f|owl| language|!||

In [78]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

async for chunk in chain.astream_events({'topic': 'parrot'}, version='v1'):
    print(chunk)

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

# Prompt + Model + Parser

In [ ]:
from langchain_core.output_parsers import StrOutputParser#ma
#

# Add retriever

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

async for chunk in retrieval_chain.astream_log(
    "where did harrison work?", include_names=["Docs"]
):
    print("-" * 40)
    print(chunk)

In [12]:
chain.astream({'topic': '

SyntaxError: unterminated string literal (detected at line 1) (1698548667.py, line 1)